In [29]:
import requests as req
import pandas as pd
from pydantic import BaseModel

In [2]:
r = req.get('https://api.dedust.io/v2/pools')
pools = pd.DataFrame(r.json()) 

In [ ]:
# edge list data (tokens)

# id;color;size
# node1;red;10
# node2;green;20
# node3;blue;30

In [20]:
r0 = pools['assets'].apply(pd.Series)[0]
r1 = pools['assets'].apply(pd.Series)[1]

In [22]:
tokens = pd.concat([r0, r1])

In [48]:
tokens.iloc[1]

{'type': 'native',
 'metadata': {'name': 'Toncoin',
  'symbol': 'TON',
  'image': 'https://assets.dedust.io/images/ton.webp',
  'decimals': 9}}

In [30]:
class Token(BaseModel):
    native:bool
    address:str
    name:str
    symbol:str

In [46]:
from typing import Any
def process_token(token: dict[str, Any]) -> Token:
    native = False
    address = ''
    name = ''
    symbol = ''

    if token.get('type') == 'native':        
        native = True
    if token.get('metadata') is not None:
        address = token['metadata'].get('address', '0x')
        name = token['metadata'].get('name')
        symbol = token['metadata'].get('symbol')
    return Token(native=native, address=address, name=name, symbol=symbol)
        

In [47]:
for token in tokens:
    print(process_token(token))

native=False address='EQBiHSLVzoicc-LQPDXJc_rF-WRft0aFihccFduA6fQ1ynYM' name='Baron' symbol='BARO'
native=True address='0x' name='Toncoin' symbol='TON'
native=True address='0x' name='Toncoin' symbol='TON'
native=False address='EQBlqsm144Dq6SjbPI4jjZvA1hqTIP3CvHovbIfW_t-SCALE' name='Scaleton' symbol='SCALE'
native=True address='0x' name='Toncoin' symbol='TON'


ValueError: Invalid data types in token information

In [42]:
tokens.iloc[5]

{'type': 'jetton',
 'address': 'EQBAeSKF9-yd2ygCrs-xNiPhNb7CtDj2ewVafmEmdMoA5v_1',
 'metadata': None}